# Cortes
>$n$: Quantidade de tipos de itens  
>$m$: Quantidade de padrões de corte  

## Variáveis
>$x_j$: Quantidade que será utilizada do padrão de corte $j$, para $j = 1,\ldots, m$  
>$s_j = 
  \begin{cases}
    1 : \text{Caso o padrão } j \text{ seja utilizado} \\
    0 : \text{Caso não seja}
  \end{cases}
$

## Parâmetros
>$D_i$: Demanda do item $i$, para $i = 1,\ldots, n$  
>$P_{ij}$: Padrão de corte $i$ do tipo de item $j$, para $i = 1,\ldots, n$ e $j = 1,\ldots, m$  
>$R$: Máximo permitido de padrões de corte  

## Modelo  

\begin{align*}
    \hbox{min} \ \ 
        & \sum_{j=1}^m x_i \\
        
    \hbox{s.a.} \ \ 
        & \sum_{j=1}^m P_{ij} x_j \geq D_i && i = 1, \ldots, n \\
        & \sum_{j=1}^m s_j \leq R                              \\
        & x_j \leq M s_j                   && j = 1, \ldots, m \\
        & M = \sum_{i=1}^n D_i                                 \\
        & s_j \in \{0, 1\}                 && j = 1, \ldots, m \\
        & x_{j} \in +\mathbb{Z}            && j = 1, \ldots, m \\
\end{align*}

In [1]:
from itertools import product


def cut_patterns(M: int, C: list[int]) -> list[list[int]]:
    corte_minimo = M - min(C)
    cortes = product(*(range(M//i + 1) for i in C))
    x = [i for i in cortes if corte_minimo < sum(a*b for a, b in zip(i, C)) <= M]

    return x

In [2]:
import pandas as pd
from IPython.display import display

def create_table(M: int, C: list[int]):
    patterns = cut_patterns(M, C)
    df = pd.DataFrame(patterns, columns=C)
    df["Resto"] = M - df.mul(C).sum(axis=1)
    # display(df)
    
    return df.to_string()

In [3]:
from pymprog import *


def modelo_cortes(M, R, C, D):
    P = [*zip(*cut_patterns(M, C))]

    n = len(D)
    m = len(P[0])

    begin("cortes")

    x = var("x", m, int)
    s = var("s", m, bool)

    minimize( sum(x) )

    for i in range(n):
        sum(P[i][j] * x[j] for j in range(m)) >= D[i]

    M = sum(D)
    for j in range(m):
        x[j] <= s[j] * M

    sum(s) <= R

    solver(int, tm_lim= 120_000)
    solve()

    obj = vobj()

    end()

    return obj, x, P

In [4]:
import os
import pandas as pd

resumo = pd.DataFrame(columns=[
    "Tamanho da matriz",
    "Qtn. de itens menores",
    "Qtn. de padrões de corte gerados",
    "Limite de Padrões de corte",
    "Qtn. de padrões de corte utilizados",
    "Total de matrizes usadas",
    "Qtn. de papel desperdiçada"
])

for file in os.listdir("./input/"):
    with open(f"input/{file}", "r") as f:
        M = int(f.readline())

        R = int(f.readline())

        C, D = [], []
        for i in f.readlines():
            c, d = map(int, i.split())
            C.append(c)
            D.append(d)

        print(f"\n{file:=^75}")

        # create_table(M, C)
        obj, varx, P = modelo_cortes(M, R, C, D)
        P = [*zip(*P)]

        with open(f"output_b/{file}", "w") as f:
            f.write(create_table(M, C) + "\n\n")

            f.write(f"Tamanho da bobina matriz: {M}\n")
            f.write(f"Tamanho das bobinas menores: {C}\n")
            f.write(f"Demandas: {D}\n\n")

            f.write(f"Quantidade de bobinas usadas: {int(obj)}\n\n")

            desperdicio_total = 0

            for i, x in enumerate(varx):
                if x.primal:
                    resto = M - sum(a*b for a, b in zip(P[i], C))
                    desperdicio_total += int(x.primal) * resto

                    f.write(f"Padrão {i:>2}: {P[i]} resto {resto}\n")
                    f.write(f"Quantidade usada: {int(x.primal)}\n")
                    f.write(f"Quantidade desperdiçada: {int(x.primal) * resto}\n")
                    f.write(f"Quantidade satisfeita de cada demanda: {tuple(j * int(x.primal) for j in P[i])}\n\n")

            f.write(f"Quantidade total de papel gasto: {int(obj) * M}\n")
            f.write(f"Quantidade desperdiçada: {desperdicio_total}\n")

            resumo.loc[file] = [
                M,
                len(C),
                len(cut_patterns(M, C)),
                R,
                sum(bool(x.primal) for x in varx),
                int(obj),
                desperdicio_total
            ]


==============================inst_200_5.txt===============================
GLPK Simplex Optimizer 5.0
57 rows, 102 columns, 282 non-zeros
      0: obj =   0.000000000e+00 inf =   1.209e+04 (5)
     13: obj =   3.446229167e+03 inf =   0.000e+00 (0)
*    20: obj =   3.338166667e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Integer Optimizer 5.0
57 rows, 102 columns, 282 non-zeros
102 integer variables, 51 of which are binary
Integer optimization begins...
Long-step dual simplex will be used
+    20: mip =     not found yet >=              -inf        (1; 0)
+  1480: >>>>>   4.689000000e+03 >=   3.339000000e+03  28.8% (1206; 0)
+  1580: >>>>>   4.201000000e+03 >=   3.339000000e+03  20.5% (1296; 2)
+  1610: >>>>>   3.942000000e+03 >=   3.339000000e+03  15.3% (1204; 189)
Solution found by heuristic: 3582
+  2433: >>>>>   3.412000000e+03 >=   3.339000000e+03   2.1% (1237; 522)
+ 34652: mip =   3.412000000e+03 >=   3.339000000e+03   2.1% (11330; 4989)
+ 45984: mip =   3.412000000

In [5]:
display(
    resumo
        # .style.set_table_styles([
        #     {'selector': '.col0', 'props': [('max-width', '80px')]},
        #     # {'selector': '.col1', 'props': [('max-width', '100px')]},
        #     # {'selector': '.col2', 'props': [('max-width', '100px')]},
        #     {'selector': '.col3', 'props': [('max-width', '120px')]},
        #     {'selector': '.col4', 'props': [('max-width', '120px')]},
        #     {'selector': '.col5', 'props': [('max-width', '130px')]},
        #     {'selector': '.col6', 'props': [('max-width', '90px')]},
        #     {'selector': '.col7', 'props': [('max-width', '100px')]},
        # ])
)

print(resumo.to_latex())

,Tamanho da matriz,Qtn. de itens menores,Qtn. de padrões de corte gerados,Limite de Padrões de corte,Qtn. de padrões de corte utilizados,Total de matrizes usadas,Qtn. de papel desperdiçada
inst_200_5.txt,200,5,51,5,5,3364,20204
inst_130_6.txt,130,6,48,7,7,5135,5956
inst_200_4.txt,200,4,14,4,3,3381,13826
inst_110_5.txt,110,5,229,5,5,1078,0
inst_100_4.txt,100,4,24,5,4,2194,10943
inst_250_5.txt,250,5,64,5,5,3084,11108
inst_100_6.txt,100,6,91,6,6,3128,6107
inst_130_4.txt,130,4,14,5,5,3081,6957
inst_250_6.txt,250,6,119,7,7,3090,2998
inst_200_6.txt,200,6,87,6,6,4084,15757


\begin{tabular}{lrrrrrrr}
\toprule
 & Tamanho da matriz & Qtn. de itens menores & Qtn. de padrões de corte gerados & Limite de Padrões de corte & Qtn. de padrões de corte utilizados & Total de matrizes usadas & Qtn. de papel desperdiçada \\
\midrule
inst_200_5.txt & 200 & 5 & 51 & 5 & 5 & 3364 & 20204 \\
inst_130_6.txt & 130 & 6 & 48 & 7 & 7 & 5135 & 5956 \\
inst_200_4.txt & 200 & 4 & 14 & 4 & 3 & 3381 & 13826 \\
inst_110_5.txt & 110 & 5 & 229 & 5 & 5 & 1078 & 0 \\
inst_100_4.txt & 100 & 4 & 24 & 5 & 4 & 2194 & 10943 \\
inst_250_5.txt & 250 & 5 & 64 & 5 & 5 & 3084 & 11108 \\
inst_100_6.txt & 100 & 6 & 91 & 6 & 6 & 3128 & 6107 \\
inst_130_4.txt & 130 & 4 & 14 & 5 & 5 & 3081 & 6957 \\
inst_250_6.txt & 250 & 6 & 119 & 7 & 7 & 3090 & 2998 \\
inst_200_6.txt & 200 & 6 & 87 & 6 & 6 & 4084 & 15757 \\
inst_130_5.txt & 130 & 5 & 38 & 5 & 5 & 3468 & 2613 \\
inst_100_5.txt & 100 & 5 & 40 & 6 & 6 & 2678 & 12879 \\
inst_250_4.txt & 250 & 4 & 25 & 4 & 4 & 2451 & 22968 \\
\bottomrule
\end{tabular}

